Test with lidar odometry and GPS only at chimera authentications

In [ ]:
import numpy as np
import struct 
import os
import plotly.graph_objects as go
import time
from tqdm import tqdm

import chimera_fgo.general as general
from chimera_fgo.io import read_lidar_bin, read_gt
from chimera_fgo.kitti_util import process_kitti_gt, load_icp_results
from chimera_fgo.geom_util import euler_to_R

import symforce
try:
    symforce.set_epsilon_to_symbol()
except symforce.AlreadyUsedEpsilon:
    print("Already set symforce epsilon")
    pass 

import symforce.symbolic as sf

%load_ext autoreload
%autoreload 2

np.set_printoptions(suppress=True, precision=3)

### Load data

In [ ]:
kitti_seq = '0034'
start_idx = 0

gtpath = os.path.join(os.getcwd(), '..', '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)

data_path = os.path.join(os.getcwd(), '..', '..', 'data', 'kitti', kitti_seq, 'results', 'p2pl_icp')
lidar_Rs, lidar_ts, positions, lidar_covariances = load_icp_results(data_path, start_idx=start_idx)

### Parameters

In [ ]:
TRAJLEN = 2000

MAX_BIAS = 100  # [m]
ATTACK_START = TRAJLEN // 2

# Sliding window of size 100 which shifts by 10 each iteration
N_SHIFT = 10
N_WINDOW = 100

GPS_RATE = 10  # ratio of odometry to GPS measurements

PR_SIGMA = 6

# Lidar odometry covariance
ODOM_R_SIGMA = 0.01  # 
ODOM_T_SIGMA = 0.05  # [m]
# #ODOM_SIGMA = np.array([0.05, 0.05, 0.05, 0.2, 0.2, 0.2])
ODOM_SIGMA = np.ones(6)
ODOM_SIGMA[:3] *= ODOM_R_SIGMA
ODOM_SIGMA[3:] *= ODOM_T_SIGMA

In [ ]:
#lidar_sigmas = [sf.Matrix(lidar_covariances[i]) for i in range(len(lidar_covariances))]

LIDAR_SIGMA = np.eye(6)
LIDAR_SIGMA[:3, :3] *= ODOM_R_SIGMA
LIDAR_SIGMA[3:, 3:] *= ODOM_T_SIGMA
lidar_sigmas = [sf.Matrix(LIDAR_SIGMA) for i in range(len(lidar_covariances))]

### Problem Setup 

In [ ]:
# Satellite positions
# Uniformly arranged in a circle of radius R at constant altitude
R = 1e4
SAT_ALT = 1e4
N_SATS = 10
satpos_enu = np.zeros((N_SATS, 3))
satpos_enu[:,0] = np.cos(np.linspace(0, 2*np.pi, N_SATS, endpoint=False)) * R
satpos_enu[:,1] = np.sin(np.linspace(0, 2*np.pi, N_SATS, endpoint=False)) * R
satpos_enu[:,2] = SAT_ALT

# Lidar odometry
lidar_odom = [None] * TRAJLEN
for i in range(TRAJLEN - 1):
    lidar_odom[i] = (lidar_Rs[i], lidar_ts[i])

### Spoofing

In [ ]:
from scipy.stats import chi2

# Compute threshold
alpha = 0.001  # False alarm (FA) rate
T = chi2.ppf(1-alpha, df=N_SATS*(N_WINDOW/GPS_RATE))
print("Threshold = ", T)

In [ ]:
# Spoofed trajectory
spoofed_pos = gt_enu[:TRAJLEN].copy()

gps_spoofing_biases = np.zeros(TRAJLEN)  
gps_spoofing_biases[ATTACK_START:] = np.linspace(0, MAX_BIAS, TRAJLEN-ATTACK_START)  # Ramping attack

spoofed_pos[:,0] += gps_spoofing_biases

# Spoofed ranges
PR_SIGMA = 6
spoofed_ranges = np.zeros((TRAJLEN, N_SATS))
for i in range(TRAJLEN):
    for j in range(N_SATS):
        spoofed_ranges[i,j] = np.linalg.norm(spoofed_pos[i] - satpos_enu[j]) + np.random.normal(0, PR_SIGMA)

In [ ]:
from lgchimera.symforce.factor_graph import fgo

graph_positions = np.zeros((TRAJLEN, 3))
init_poses = [sf.Pose3.identity()] * N_WINDOW
init_poses[0] = sf.Pose3(sf.Rot3.from_rotation_matrix(gt_Rs[0]), sf.V3(gt_enu[0]))
e_ranges = np.zeros((N_WINDOW//GPS_RATE, N_SATS))
qs = np.zeros(TRAJLEN//GPS_RATE)

for k in (pbar := tqdm(range((TRAJLEN - N_WINDOW) // N_SHIFT))):
    #print(N_SHIFT * k, '/', TRAJLEN)
    window = slice(N_SHIFT * k, N_SHIFT * k + N_WINDOW)
    odom = lidar_odom[window]
    ranges = spoofed_ranges[window]
    #odom_sigmas = lidar_sigmas[window]
    # if k == 180:
    #     gps_rate = 50
    # else:
    #     gps_rate = 100

    result = fgo(init_poses, satpos_enu, ranges, odom, PR_SIGMA, ODOM_SIGMA, gps_rate, fix_first_pose=True, debug=False)

    # Extract optimized positions
    window_positions = np.zeros((N_WINDOW, 3))
    for i in range(N_WINDOW):
        pose = result.optimized_values["poses"][i]
        window_positions[i] = pose.position().flatten()   

    # Save trajectory
    graph_positions[N_SHIFT*k:N_SHIFT*(k+1)] = window_positions[:N_SHIFT]

    # Update initial poses
    init_poses[:-N_SHIFT] = result.optimized_values["poses"][N_SHIFT:]

    # Use lidar odometry to initialize latest new poses
    n_pose = result.optimized_values["poses"][-1]
    n_pose = sf.Pose3(sf.Rot3.from_rotation_matrix(n_pose.R.to_rotation_matrix()), sf.V3(n_pose.t))
    for i in range(N_SHIFT):
        lidar_T = sf.Pose3(sf.Rot3.from_rotation_matrix(lidar_Rs[N_SHIFT*k + N_WINDOW + i]), sf.V3(lidar_ts[N_SHIFT*k + N_WINDOW + i]))
        n_pose = lidar_T * n_pose
        init_poses[-N_SHIFT+i] = n_pose

    # Compute test statistic
    # for i in range(N_WINDOW//GPS_RATE):
    #     for j in range(N_SATS):
    #         e_ranges[i,j] = np.linalg.norm(window_positions[::GPS_RATE][i] - satpos_enu[j])
    # q = np.sum((ranges[::GPS_RATE] - e_ranges)**2) / PR_SIGMA**2
    # #print("q = ", q)
    # #print(N_SHIFT * k, '/', TRAJLEN, "q = ", q)
    # pbar.set_description(f"q = {q:.2f}")
    # qs[k] = q

    # Mitigation
    # if q > T:
    #     print("Attack detected at ", N_SHIFT * k)
    #     PR_SIGMA = 1000

# # End of trajectory, save all the positions from the current window
# for i in range(N_WINDOW):
#     idx = TRAJLEN - N_WINDOW + i
#     graph_positions[idx] = result.optimized_values["poses"][i].position().flatten()
OPT_TRAJLEN = N_SHIFT*(k+1)

In [ ]:
# For plotting section of q
qs_orig = qs
qs = qs_orig[:133].copy()

In [ ]:
OPT_TRAJLEN = N_SHIFT*(k+1)
graph_positions = graph_positions[:OPT_TRAJLEN]
gt_enu = gt_enu[:OPT_TRAJLEN]
qs = qs[:OPT_TRAJLEN//N_SHIFT]

In [ ]:
# Plot test statistic and threshold
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(qs)), y=qs, name='Test statistic'))
fig.add_trace(go.Scatter(x=np.arange(len(qs)), y=T*np.ones(len(qs)), name='Threshold', line=dict(color='red', dash='dash')))
# Add vertical line at start of spoofing attack
fig.add_shape(type="line", x0=ATTACK_START/GPS_RATE, y0=-20, x1=ATTACK_START/GPS_RATE, y1=200, line=dict(color="black", width=2, dash="dash"))
fig.update_layout(width=900, height=500, xaxis_title='Time [s]', yaxis_title='Test statistic')
#fig.update_layout(legend=dict(x=0.75, y=0.98), font=dict(size=18))
fig.show()

In [ ]:
fgo_traj = go.Scatter(x=graph_positions[:,0], y=graph_positions[:,1], hovertext=np.arange(TRAJLEN), name='FGO trajectory')
gt_traj = go.Scatter(x=gt_enu[:TRAJLEN,0], y=gt_enu[:TRAJLEN,1], hovertext=np.arange(TRAJLEN), name='Ground-truth')
spoof_traj = go.Scatter(x=spoofed_pos[:OPT_TRAJLEN,0], y=spoofed_pos[:OPT_TRAJLEN,1], hovertext=np.arange(OPT_TRAJLEN), name='Spoofed')
start = go.Scatter(x=[0], y=[0], name='Start', mode='markers', marker=dict(size=10, color='blue'), showlegend=False)
fig = go.Figure(data=[gt_traj, fgo_traj, spoof_traj, start])
fig.update_layout(width=700, height=700, xaxis_title='East [m]', yaxis_title='North [m]')
# Move legend into plot
# fig.update_layout(legend=dict(x=0.75, y=0.98), font=dict(size=18))
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
fig.update_xaxes(autorange=True)
fig.show()

In [ ]:
# Plot x and y errors over time
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(TRAJLEN), y=graph_positions[:TRAJLEN,0] - gt_enu[:TRAJLEN,0], name='x error'))
fig.add_trace(go.Scatter(x=np.arange(TRAJLEN), y=graph_positions[:TRAJLEN,1] - gt_enu[:TRAJLEN,1], name='y error'))
fig.add_trace(go.Scatter(x=np.arange(TRAJLEN), y=graph_positions[:TRAJLEN,2] - gt_enu[:TRAJLEN,2], name='z error'))
# Plot RMSE
fig.add_trace(go.Scatter(x=np.arange(TRAJLEN), y=np.sqrt(np.mean((graph_positions - gt_enu[:TRAJLEN])**2, axis=1)), name='RMSE'))
fig.update_layout(width=900, height=500, xaxis_title='Time [s]', yaxis_title='Error [m]')
fig.update_layout(legend=dict(x=0.75, y=0.98), font=dict(size=18))
fig.show()


In [ ]:
# RMSE error
rmse_xyz = np.sqrt(np.mean((graph_positions[:1800] - gt_enu[:1800])**2, axis=0))
print("RMSE (xyz): ", rmse_xyz)
print("RMSE (overall): ", rmse_xyz.mean())

In [ ]:
# mean error
np.mean((graph_positions[:1800] - gt_enu[:1800]), axis=0)